<a href="https://colab.research.google.com/github/neuralguy18/demo/blob/main/Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Self attention is a mechanism in transformers used to compute more efficient input representations by allowing each position in a sequence to interact with and weigh the importance of all other positions within the same sequence.


In self-attention,our goal is to calculate context vectors z(i) for each element x(i) in the input sequence. A context vector can be interpreted as an enriched embedding vector.


In [1]:
import torch
inputs = torch.tensor(
    [[0.43,0.15,0.89],
     [0.55,0.87,0.66],
     [0.57,0.85,0.64],
     [0.22,0.58,0.33],
     [0.77,0.25,0.10],
     [0.05,0.8,0.55]]
)

In [2]:
query = inputs[1]
attn_scores_2 = torch.empty(inputs.shape[0])
for i,xi in enumerate(inputs):
  attn_scores_2[i] = torch.dot(query,xi)
attn_scores_2

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])

In [3]:
attn_weights_2_temp = torch.exp(attn_scores_2) / torch.exp(attn_scores_2).sum(dim=0)
attn_weights_2_temp

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [4]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i,xi in enumerate(inputs):
  context_vec_2 += attn_weights_2_temp[i]*xi
context_vec_2

tensor([0.4419, 0.6515, 0.5683])

In [5]:
attn_scores = torch.empty(6,6)
for i,xi in enumerate(inputs):
  for j,xj in enumerate(inputs):
    attn_scores[i,j] = torch.dot(xi,xj)
attn_scores


tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [6]:
attn_scores = inputs @ inputs.T
attn_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [7]:
attn_weights = torch.softmax(attn_scores,dim=1)
attn_weights

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [8]:
context_vectors = attn_weights @ inputs
context_vectors

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

In [9]:
context_vec_2

tensor([0.4419, 0.6515, 0.5683])

We will implement the self-attention mechanism step by step by introducing the three trainable weight matrices Wq, Wk and Wv


In [10]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2


In [11]:
torch.manual_seed(123)
Wq = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
Wk = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)
Wv = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad=False)


In [12]:
query_2 = x_2 @ Wq
key_2 = x_2 @ Wk
value_2 = x_2 @ Wv
query_2,key_2,value_2

(tensor([0.4306, 1.4551]), tensor([0.4433, 1.1419]), tensor([0.3951, 1.0037]))

In [13]:
keys = inputs @ Wk
values = inputs @ Wv
keys,values

(tensor([[0.3669, 0.7646],
         [0.4433, 1.1419],
         [0.4361, 1.1156],
         [0.2408, 0.6706],
         [0.1827, 0.3292],
         [0.3275, 0.9642]]),
 tensor([[0.1855, 0.8812],
         [0.3951, 1.0037],
         [0.3879, 0.9831],
         [0.2393, 0.5493],
         [0.1492, 0.3346],
         [0.3221, 0.7863]]))

In [14]:
keys_2 = keys[1]
attn_score_22= query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


In [15]:
attn_scores_2 = query_2 @ keys.T
attn_scores_2

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])

In [16]:
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2/d_k**0.5,dim=-1)
attn_weights_2

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])

In [17]:
context_vec_2 = attn_weights_2 @ values
context_vec_2

tensor([0.3061, 0.8210])

In [18]:
import torch.nn as nn
class SelfAttention_v1(nn.Module):
  def __init__(self,d_in,d_out):
    super().__init__()
    self.W_query = nn.Parameter(torch.rand(d_in,d_out),requires_grad=True)
    self.W_key = nn.Parameter(torch.rand(d_in,d_out),requires_grad=True)
    self.W_value = nn.Parameter(torch.rand(d_in,d_out),requires_grad=True)

  def forward(self,x):
    keys = x @ self.W_key
    values = x @ self.W_value
    query = x @ self.W_query
    attn_scores = query @ keys.T
    attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5,dim=-1)
    context_vectors = attn_weights @ values
    return context_vectors


In [19]:
torch.manual_seed(123)
self_attn_v1 = SelfAttention_v1(d_in,d_out)
context_vectors_v1 = self_attn_v1(inputs)
context_vectors_v1

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)

In [20]:
class SelfAttention_v2(nn.Module):
  def __init__(self,d_in,d_out,qkv_bias=False):
    super().__init__()
    self.W_query = nn.Linear(d_in,d_out,bias=qkv_bias)
    self.W_key = nn.Linear(d_in,d_out,bias=qkv_bias)
    self.W_value = nn.Linear(d_in,d_out,bias=qkv_bias)

  def forward(self,x):
    keys = self.W_key(x)
    values = self.W_value(x)
    queries = self.W_query(x)
    attn_scores = queries @ keys.T
    attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5,dim=-1)
    context_vectors = attn_weights @ values
    return context_vectors


In [21]:
torch.manual_seed(789)
self_attn_v2 = SelfAttention_v2(d_in,d_out)
context_vectors_v2 = self_attn_v2(inputs)
context_vectors_v2

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)

Causal attention is a specialized form of self-attention. It restricts a model to only consider previous and current inputs in a sequence when processing any token when computing attention scores.

In [22]:
queries = self_attn_v2.W_query(inputs)
keys = self_attn_v2.W_key(inputs)
values = self_attn_v2.W_value(inputs)
attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5,dim=-1)
attn_weights

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)

In [23]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length,context_length))
mask_simple

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])

In [24]:
masked_simple = attn_weights * mask_simple
masked_simple

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)

In [25]:
row_sums = masked_simple.sum(dim=-1,keepdim=True)
masked_simple_norm = masked_simple / row_sums
masked_simple_norm


tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)

In [26]:
mask = torch.triu(torch.ones(context_length,context_length),diagonal=1)
mask
masked = attn_scores.masked_fill(mask.bool(),-float('inf'))
masked


tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)

In [27]:
attn_weights_masked = torch.softmax(masked/keys.shape[1]**0.5,dim =1)
attn_weights_masked

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)

In [28]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
example = torch.ones(6,6)
example
print(dropout(example))

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


In [29]:
torch.manual_seed(123)
dropout(attn_weights_masked)


tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.8966, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4921, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4350, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)

In [30]:
batch = torch.stack((inputs,inputs),dim=0)
print(batch.shape)

torch.Size([2, 6, 3])


In [31]:
class CausalAttention(nn.Module):
  def __init__(self,d_in,d_out,context_length,dropout, qkv_bias=False):
    super().__init__()
    self.d_in = d_in
    self.d_out = d_out
    self.W_query = nn.Linear(d_in,d_out,bias=qkv_bias)
    self.W_key = nn.Linear(d_in,d_out,bias=qkv_bias)
    self.W_value = nn.Linear(d_in,d_out,bias=qkv_bias)
    self.dropout = nn.Dropout(dropout)
    self.register_buffer("mask",torch.triu(torch.ones(context_length,context_length),diagonal=1))

  def forward(self,x):
    b,num_tokens,d_in = x.shape
    queries = self.W_query(x)
    keys = self.W_key(x)
    values = self.W_value(x)

    attn_scores = queries @ keys.transpose(1,2)
    attn_scores = attn_scores.masked_fill(self.mask.bool()[:num_tokens, :num_tokens],-torch.inf)
    attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5,dim=-1)
    attn_weights = self.dropout(attn_weights)
    context_vectors = attn_weights @ values
    return context_vectors



In [32]:
torch.manual_seed(123)
context_length= batch.shape[1]
causal_attn = CausalAttention(d_in,d_out,context_length,0.5)
context_vectors_causal = causal_attn(batch)
context_vectors_causal

tensor([[[ 0.0000,  0.0000],
         [-0.4368,  0.2142],
         [-0.7751,  0.0077],
         [-0.9140, -0.2769],
         [ 0.0000,  0.0000],
         [-0.6906, -0.0974]],

        [[-0.9038,  0.4432],
         [ 0.0000,  0.0000],
         [-0.2883,  0.1414],
         [-0.9140, -0.2769],
         [-0.4416, -0.1410],
         [-0.5272, -0.1706]]], grad_fn=<UnsafeViewBackward0>)

In [33]:
context_vectors_causal.shape

torch.Size([2, 6, 2])

In [34]:
class MultiHeadAttentionWrapper(nn.Module):
  def __init__(self,d_in,d_out,num_heads,context_length,dropout,qkv_bias=False):
    super().__init__()
    self.heads = nn.ModuleList(
        [CausalAttention(d_in,d_out,context_length,dropout,qkv_bias) for _ in range(num_heads)]
    )

  def forward(self,x):
    return torch.cat([head(x) for head in self.heads],dim=-1)

In [35]:
torch.manual_seed(123)
context_length = batch.shape[1]
multihead_attn = MultiHeadAttentionWrapper(d_in,d_out,4,context_length,0.5)
context_vectors_multihead = multihead_attn(batch)
context_vectors_multihead

tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000, -1.1367,
           1.0127],
         [-0.7381, -0.2026,  0.4199,  0.0936,  1.1584,  0.6021, -0.6394,
           0.5696],
         [-0.7751,  0.0077,  1.2404,  0.7719,  0.0000,  0.0000,  0.0000,
           0.0000],
         [-0.7714, -0.0687,  1.0955,  0.7178,  0.2413,  0.1442, -0.5586,
           0.6790],
         [-0.8109, -0.1154,  0.7901,  0.4852,  0.7609,  0.3158, -0.2946,
           0.5063],
         [-0.2728,  0.0205,  0.2324,  0.1700,  0.6268,  0.3097, -0.6360,
           0.9256]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.9131,  0.5458, -1.1367,
           1.0127],
         [-0.7381, -0.2026,  0.7582,  0.5577,  0.4965,  0.2967, -0.6394,
           0.5696],
         [-1.2600, -0.1264,  0.2701,  0.0602,  0.4538,  0.2094, -0.2988,
           0.4935],
         [ 0.0000,  0.0000,  0.2161,  0.0481,  0.0000,  0.0000, -0.4776,
           0.7866],
         [-0.7775, -0.2243,  0.1816,  0.0405,  0.3638,  0.1352, -0.6

In [38]:
class MultiHeadAttention(nn.Module):
  def __init__(self,d_in,d_out,context_length,dropout,num_heads,qkv_bias=False):
    super().__init__()
    assert(d_out % num_heads == 0), "d_out must be divisible by num_heads"
    self.d_out = d_out
    self.num_heads = num_heads
    self.head_dim = d_out // num_heads
    self.W_query = nn.Linear(d_in,d_out,bias=qkv_bias)
    self.W_key = nn.Linear(d_in,d_out,bias=qkv_bias)
    self.W_value = nn.Linear(d_in,d_out,bias=qkv_bias)
    self.out_proj = nn.Linear(d_out,d_out)
    self.dropout = nn.Dropout(dropout)
    self.register_buffer("mask",torch.triu(torch.ones(context_length,context_length),diagonal=1))

  def forward(self,x):
    b,num_tokens,d_in = x.shape
    queries = self.W_query(x)
    keys = self.W_key(x)
    values = self.W_value(x)

    keys = keys.view(b,num_tokens,self.num_heads,self.head_dim)
    values = values.view(b,num_tokens,self.num_heads,self.head_dim)
    queries = queries.view(b,num_tokens,self.num_heads,self.head_dim)

    keys = keys.transpose(1,2)
    values = values.transpose(1,2)
    queries = queries.transpose(1,2)

    attn_scores = queries @ keys.transpose(2,3)
    mask_bool = self.mask.bool()[:num_tokens,:num_tokens]
    attn_scores = attn_scores.masked_fill(mask_bool,-torch.inf)

    attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5,dim=-1)
    attn_weights = self.dropout(attn_weights)
    context_vectors = ((attn_weights @ values).transpose(1,2)).contiguous().view(
                                      b,num_tokens,self.d_out)
    context_vectors = self.out_proj(context_vectors)
    return context_vectors





In [39]:
torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in,d_out,context_length,0.0,num_heads = 2)
context_vectors_mha = mha(batch)
context_vectors_mha


tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)